In [3]:
import sys
import os
from datetime import datetime, timedelta, timezone
import requests
from zoneinfo import ZoneInfo
from collections import defaultdict, deque
import math
from pprint import pprint
from operator import itemgetter

# Add the project root directory to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../app"))
if project_root not in sys.path:
    sys.path.append(project_root)

from query import Query
from metrics import Metrics

In [11]:
# Use last season stats so there aren't any spoilers
# 2021 NHL 411 55842
current = False
league_id = "55842" # "67269" if current else "97108"
game_id = 411 # 453 if current else 427
league_key = f"{game_id}.l.{league_id}"
# league_key = "386.l.88439"
query = await Query.create(league_key)

[2025-04-04 17:12:12,650 DEBUG] [yahoo_oauth.oauth.handler] AUTHORIZATION URL : https://api.login.yahoo.com/oauth2/request_auth?redirect_uri=oob&response_type=code&client_id=dj0yJmk9bDMxZTVhUzhXazlFJmQ9WVdrOU9FaHNlbnBvV1dVbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTU4
[2025-04-04 17:12:18,137 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2025-04-04 17:12:18,242 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 0.10391497611999512
[2025-04-04 17:12:18,242 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2025-04-04 17:12:19,393 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1.2553410530090332
[2025-04-04 17:12:19,394 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x112e00590>
[2025-04-04 17:12:26,251 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 8.11349081993103
[2025-04-04 17:12:26,252 DEBUG] [yahoo_oauth.oauth.token_is_valid] T

In [10]:
url = f"/games;seasons=2021;game_codes=nhl"
response = await query.get_response(url)
response


[2025-04-04 17:10:58,707 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 134.52820682525635
[2025-04-04 17:10:58,741 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


{'games': {'game': {'game_key': '411',
   'game_id': '411',
   'name': 'Hockey',
   'code': 'nhl',
   'type': 'full',
   'url': 'https://hockey.fantasysports.yahoo.com/2021/hockey',
   'season': '2021',
   'is_registration_over': '1',
   'is_game_over': '1',
   'is_offseason': '1'}}}

In [13]:
query.league_start_date_str

'2021-10-12'

In [12]:
url = f"/league/{query.league_key}/transactions"
response = await query.get_response(url)
response["league"]["transactions"]

# drops = {}
# missed = 0
# for transaction in transactions:

[2025-04-04 17:12:49,257 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 31.119323015213013
[2025-04-04 17:12:49,337 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


[{'transaction_key': '411.l.55842.tr.586',
  'transaction_id': '586',
  'type': 'add/drop',
  'status': 'successful',
  'timestamp': '1650174285',
  'players': [{'player_key': '411.p.6445',
    'player_id': '6445',
    'name': {'full': 'Elvis Merzlikins',
     'first': 'Elvis',
     'last': 'Merzlikins',
     'ascii_first': 'Elvis',
     'ascii_last': 'Merzlikins'},
    'editorial_team_abbr': 'CBJ',
    'display_position': 'G',
    'position_type': 'G',
    'transaction_data': {'type': 'add',
     'source_type': 'freeagents',
     'destination_type': 'team',
     'destination_team_key': '411.l.55842.t.9',
     'destination_team_name': 'Nathan’s McBack again'}},
   {'player_key': '411.p.6405',
    'player_id': '6405',
    'name': {'full': 'Thatcher Demko',
     'first': 'Thatcher',
     'last': 'Demko',
     'ascii_first': 'Thatcher',
     'ascii_last': 'Demko'},
    'editorial_team_abbr': 'VAN',
    'display_position': 'G',
    'position_type': 'G',
    'transaction_data': {'type': 'dr

In [ ]:
metrics = Metrics(query)

In [ ]:
await metrics.get_draft_busts_steals()

In [ ]:
await query.get_response(f"/league/{league_key}")

In [ ]:
query.player_points_by_date = {}

In [ ]:
async def get_roster_by_team_and_date():
    # Get roster data from draft and transactions
    teams = query.get_teams()
    url = f"/league/{query.league_key}/draftresults"
    response = await query.get_response(url)
    draft_results = response["league"]["draft_results"]
    rosters = {team_key: defaultdict(set) for team_key in teams} # by team and date
    cur_date = query.league_start_date_str
    for draft_pick in draft_results:
        rosters[draft_pick["team_key"]][cur_date].add(draft_pick["player_key"])
    url = f"/league/{query.league_key}/transactions"
    response = await query.get_response(url)
    transactions = list(reversed(response["league"]["transactions"]))
    # transaction_types = set(transaction["type"] for transaction in transactions for player in transaction["players"] if players in transaction)
    # pprint(transactions)
    for transaction in transactions:
        cur_date = datetime.fromtimestamp(int(transaction["timestamp"])).strftime("%Y-%m-%d")
        if not "players" in transaction:
            continue
        for player in transaction["players"]:
            for team_key in [player["transaction_data"].get("destination_team_key", None), player["transaction_data"].get("source_team_key", None)]:
                if team_key in rosters and cur_date not in rosters[team_key]:
                    rosters[team_key][cur_date] = rosters[team_key][list(rosters[team_key].keys())[-1]].copy()
            if player["transaction_data"]["type"] in ["add", "trade"]:
                rosters[player["transaction_data"]["destination_team_key"]][cur_date].add(player["player_key"])
            if player["transaction_data"]["type"] in ["drop", "trade"]:
                rosters[player["transaction_data"]["source_team_key"]][cur_date].remove(player["player_key"])
    return rosters

In [ ]:
async def get_players_points_from_date(player_keys, start_date_str):
    query_player_keys = []
    for player_key in player_keys:
        if player_key not in query.player_points_by_date:
            query_player_keys.append(player_key)
    dates = [(datetime.strptime(start_date_str, "%Y-%m-%d") + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((datetime.strptime(query.league_end_date_str, "%Y-%m-%d") - datetime.strptime(start_date_str, "%Y-%m-%d")).days + 1)]
    dates_csv = ",".join(dates)
    for i in range(0, len(query_player_keys), 25):
        player_keys_csv = ",".join(query_player_keys[i : min(i + 25, len(query_player_keys))])
        url = f"/league/{query.league_key}/players;player_keys={player_keys_csv}/stats_collection;types=date;date={dates_csv}"
        response = await query.get_response(url)
        cur_players = response["league"]["players"]
        for player in cur_players:
            query.player_points_by_date[player["player_key"]] = player

In [ ]:
def calculate_optimal_team_points_for_date(player_keys, date_str):
    """
    Params:
        
    """
    def get_player_points(player_key, date_str):
        return next((float(player_points["total"]) for player_points in query.player_points_by_date[player_key]["player_stats_collection"]["player_points"] if player_points["date"] == date_str))
    
    player_keys = [player_key for player_key in player_keys if get_player_points(player_key, date_str) > 0]
    player_keys = sorted(player_keys, key=lambda x: get_player_points(x, date_str), reverse=True)
    league_position_slots = query.league_roster_positions
    position_slots = defaultdict(set) # Set of player keys for each position
    total_points_for_date = 0
    for player_key in player_keys:
        # TODO: Eventually check if positions are full to improve performance, but need to consider leagues with different settings
        def is_roster_full():
            return all(len(position_slots.get(pos, set())) == league_position_slots[pos] for pos in league_position_slots)

        def find_swap_path(player_key):
            """
            Assigns players to roster positions in order to maximaize total roster points.
            Approaches this as a graphing problem where each position is a node and players with multiple position eligibility are edges.
            BFS is used to find the shortest path from the current player's positions to a position with an open slot.
            Params:
                player_key (string): player_key
            """
            nonlocal total_points_for_date
            queue = deque()
            visited = set()
            parent_map = {}
            # print(f'{players_dict[player_key]["name"]["full"]} {players_dict[player_key]["display_position"]}')

            for pos in query.player_points_by_date[player_key]["display_position"].split(","):
                queue.append({"player_key": player_key, "cur_pos": None, "new_pos": pos})
                visited.add(pos)
                parent_map[pos] = {"player_key": player_key, "cur_pos": None, "new_pos": pos}
            
            while queue:
                cur_player_key, cur_pos, new_pos = itemgetter("player_key", "cur_pos", "new_pos")(queue.popleft())
                # print(cur_player_key, cur_pos, new_pos)

                if (len(position_slots.get(new_pos, set())) < league_position_slots[new_pos]):
                    # If player can be fit into starting position
                    total_points_for_date += get_player_points(player_key, date_str)
                    path = []
                    while True:
                        path.append({"player_key": cur_player_key, "old_pos": cur_pos, "new_pos": new_pos})
                        if cur_pos is None:
                            break
                        cur_player_key, cur_pos, new_pos = itemgetter("player_key", "cur_pos", "new_pos")(parent_map[cur_pos])
                    path.reverse()
                    return path
                
                for assigned_player_key in position_slots[new_pos]:
                    for pos in query.player_points_by_date[assigned_player_key]["display_position"].split(","):
                        if pos not in visited:
                            # print("hello", {"player_key": assigned_player_key, "cur_pos": new_pos, "new_pos": pos})
                            queue.append({"player_key": assigned_player_key, "cur_pos": new_pos, "new_pos": pos})
                            visited.add(pos)
                            parent_map[pos] = {"player_key": assigned_player_key, "cur_pos": new_pos, "new_pos": pos}
            return None

        def swap_players(path):
            nonlocal position_slots
            for swap in path:
                position_slots[swap["new_pos"]].add(swap["player_key"])
                if swap["old_pos"] is not None:
                    position_slots[swap["old_pos"]].remove(swap["player_key"])

        if is_roster_full():
            break

        swap_path = find_swap_path(player_key)
        # print(swap_path)
        if swap_path is not None:
            swap_players(swap_path)
    # pprint({pos: [f'{players_dict[player_key]["name"]["full"]} {players_dict[player_key]["display_position"]} {players_dict[player_key]["player_stats_collection"]["player_points"][0]["total"]}' for player_key in player_keys] for pos, player_keys in position_slots.items()})
    return total_points_for_date


In [ ]:
url = f"/teams;team_keys=427.l.97108.t.1,427.l.97108.t.2/roster;date=2023-10-10/players/stats_collection;types=date;date=2023-10-10"
resp = await query.get_response(url)
players = resp["teams"]
players

In [ ]:
total_points_for_date = calculate_optimal_team_points_for_date(players, "2023-10-14")
print(total_points_for_date)

In [ ]:
url = f"/league/{query.league_key};out=standings,settings"
resp = await query.get_response(url)
pprint(resp)

In [ ]:
url = f"/team/427.l.97108.t.1/roster;date=2023-10-10,2023-10-11/players/stats;type=date;date=2023-10-10"
resp = await query.get_response(url)
pprint(resp["team"]["roster"]["players"])

In [ ]:
def get_players_for_week(week):
    """
    Get all players across all teams for a given week.
    """
    dates = query.get_dates_by_week(week)
    player_keys = set([player_key for team_key in rosters for date_str in dates if date_str in rosters[team_key] for player_key in rosters[team_key][date_str]])
    return player_keys

In [ ]:
rosters = await get_roster_by_team_and_date()
pprint(rosters)

In [ ]:
# Doesn't really account for pickuops made at the end of last week for the next week
stream_points_by_team = defaultdict(float)
for week in range(query.league_start_week, query.league_end_week + 1):
    dates_in_week = query.get_dates_by_week(week)
    players = get_players_for_week(week)
    await get_players_points_from_date(players, dates_in_week[0])
    for team in query.teams:
        roster_date_start = next((cur_date_str for cur_date_str in reversed(list(rosters[team["team_key"]].keys())) if datetime.strptime(cur_date_str, "%Y-%m-%d") <= datetime.strptime(dates_in_week[0], "%Y-%m-%d")), None)
        for date_str in dates_in_week:
            # expected_points: if no moves were made, actual_points: accounting for transactions made
            roster_date = next((cur_date_str for cur_date_str in reversed(list(rosters[team["team_key"]].keys())) if datetime.strptime(cur_date_str, "%Y-%m-%d") <= datetime.strptime(date_str, "%Y-%m-%d")), None)
            actual_roster = rosters[team["team_key"]][roster_date]
            expected_roster = rosters[team["team_key"]][roster_date_start]
            actual_points = calculate_optimal_team_points_for_date(actual_roster, date_str)
            expected_points = calculate_optimal_team_points_for_date(expected_roster, date_str)
            stream_points_by_team[team["team_key"]] += (actual_points - expected_points)

In [ ]:
[f"{query.get_team_name_from_key(team_key)} {points}" for team_key, points in stream_points_by_team.items()]

In [ ]:
for team_key, points in stream_points_by_team.items():
    print(f"{query.get_team_name_from_key(team_key)}: {points}")

In [ ]:
def is_first_day_of_week(date_str):
    return not all(date_str != game_week["start"] for game_week in query.game_weeks)

In [ ]:
def get_week_from_date(date):
    return next((int(game_week["week"]) for game_week in query.game_weeks if datetime.strptime(game_week["end"], "%Y-%m-%d") + timedelta(1) > date), None) # Add 1 to date for edge case

In [ ]:
def get_first_day_of_week(week):
    return next(datetime.strptime(game_week["start"], "%Y-%m-%d") for game_week in query.game_weeks if int(game_week["week"]) == week)

In [ ]:
# Get all transactions
url = f'{BASE_URL}/league/{query.league_key}/transactions'
transactions = list(reversed(query.get_response(url)["league"]["transactions"]))
players_daily = defaultdict(set)
players_weekly = defaultdict(set)
last_transaction_date = datetime.strptime(query.league_start_date_str, "%Y-%m-%d")
last_transaction_week = query.league_start_week
points_by_team = defaultdict(float)
for transaction in transactions[:46]:
    if not "players" in transaction:
        continue
    # pprint(transaction["players"])
    drop_player = next((player for player in transaction["players"] if player["transaction_data"]["type"] == "drop"), None)
    if drop_player is not None:
        transaction_date = datetime.fromtimestamp(int(transaction["timestamp"]))
        transaction_week = get_week_from_date(transaction_date)
        if not transaction_date.strftime("%Y-%m-%d") == last_transaction_date.strftime("%Y-%m-%d") and (transaction_week == last_transaction_week or is_first_day_of_week(transaction_date.strftime("%y-%m-%d"))):
            dates = [(last_transaction_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((transaction_date.date() - last_transaction_date.date()).days)]
            points_by_team_cur = get_players_points_by_date(players_daily, dates)
            for key, value in points_by_team_cur.items():
                points_by_team[key] += value
            players_daily[drop_player["player_key"]].add(drop_player["transaction_data"]["source_team_key"])
            last_transaction_date = transaction_date
        elif not transaction_date.strftime("%Y-%m-%d") == last_transaction_date.strftime("%Y-%m-%d") and transaction_week != last_transaction_week and not is_first_day_of_week(transaction_date.strftime("%y-%m-%d")):
            # Get daily points up until last day of that week
            week_first_date = get_first_day_of_week(transaction_week)
            dates = [(last_transaction_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((week_first_date.date() - last_transaction_date.date()).days)]
            points_by_team_cur = get_players_points_by_date(players_daily, dates)
            for key, value in points_by_team_cur.items():
                points_by_team[key] += value
            weeks = [week for week in range(last_transaction_week, transaction_week)]
            get_players_points_by_week(players_weekly, weeks)
            for key, value in players_daily.items():
                players_weekly[key].union(value)
            players_daily = defaultdict(set)
            last_transaction_date = transaction_date
            last_transaction_week = transaction_week
        players_daily[drop_player["player_key"]].add(drop_player["transaction_data"]["source_team_key"])

In [ ]:
url = f'/league/{query.league_key}/players;player_keys=427.p.5391/stats;type=week;week=1'
await query.get_response(url)

In [ ]:
url = f'{BASE_URL}/league/{query.league_key}/transactions'
transactions = list(reversed(query.get_response(url)["league"]["transactions"]))
transactions

In [ ]:
points_by_team

In [ ]:
query.get_team_name_from_key("427.l.97108.t.2")

In [ ]:
'''
Get's daily scores for each matchup of the week.
'''
def get_matchups_by_week(week):
    dates = get_dates_by_week(week)
    resp = query.get_response(
        f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/scoreboard;week={week}/matchups/teams/stats_collection;types=date;date={','.join(dates)}", 
    )
    return teams_dict

In [ ]:
def get_matchups():
    weeks = ",".join([str(week) for week in range(query.league_start_week, query.league_end_week + 1)])
    url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_key}/scoreboard;week={weeks}"
    return query.get_response(url)["league"]["scoreboard"]["matchups"]

In [ ]:
matchups = get_matchups()

In [ ]:
matchups

In [ ]:
def get_opp_team_by_week(team_key, week):
    # TODO make this a dict to improve performance so don't have to search through list every time
    opp_key = [matchup["teams"][0]["team_key"] if matchup["teams"][0]["team_key"] != team_key else matchup["teams"][1]["team_key"] for matchup in matchups if int(matchup["week"]) == week and team_key in [team["team_key"] for team in matchup["teams"]]]
    return opp_key[0] if len(opp_key) else None

In [ ]:
HITS_STAT_ID = 31
hits_by_team = defaultdict(int)
top_player_by_team = {}
opp_players_by_team = defaultdict(lambda: defaultdict(lambda: {"name": None, "image_url": None, "points": 0}))
for team in query.teams:
    team_points_by_nhl_team = defaultdict(float)
    team_points_by_player = defaultdict(lambda: {"name": None, "image_url": None, "points": 0})
    for week in range(query.league_start_week, query.league_end_week + 1):
        opp_team = get_opp_team_by_week(team["team_key"], week)
        if not opp_team:
            # Means this team had no matchup for the current week, skip
            continue
        week_end_date = query.get_dates_by_week(week)[-1]
        url = f"https://fantasysports.yahooapis.com/fantasy/v2/team/{team['team_key']}/roster;week={week}/players/stats;type=week;week={week}"
        roster_players = query.get_response(url)["team"]["roster"]["players"]
        for player in roster_players:
            # hits_by_team[team["team_key"]] += next(iter([int(stat["value"]) for stat in player["player_stats"]["stats"] if int(stat["stat_id"]) == HITS_STAT_ID and stat["value"] != "-"]), 0)
            
            if not player["player_points"]["total"]:
                # Can skip rest if no points
                continue

            # Team points by player
            # if team_points_by_player[player["player_key"]]["name"] is None:
            #     team_points_by_player[player["player_key"]]["name"] = player["name"]["full"]
            #     team_points_by_player[player["player_key"]]["image_url"] = player["image_url"]
            # team_points_by_player[player["player_key"]]["points"] += float(player["player_points"]["total"])
            
            # Team points by opposing player
            if opp_players_by_team[opp_team][player["player_key"]]["name"] is None:
                opp_players_by_team[opp_team][player["player_key"]]["name"] = player["name"]["full"]
                opp_players_by_team[opp_team][player["player_key"]]["image_url"] = player["image_url"]
            opp_players_by_team[opp_team][player["player_key"]]["points"] += float(player["player_points"]["total"])

            # Team points by NHL team
            # player_game_log = query.get_game_log_by_player(player["player_key"], player["name"]["full"], player["display_position"])
            # nhl_team = query.get_player_team_on_date(player_game_log, week_end_date)
            # team_points_by_nhl_team[nhl_team] += float(player["player_points"]["total"])
            
    # team_points_by_nhl_team = sorted(team_points_by_nhl_team.items(), key=lambda item: item[1], reverse=True)
    # team_points_by_player = sorted(team_points_by_player.values(), key=lambda value: value["points"], reverse=True)
    # top_nhl_team = team_points_by_nhl_team[0][0]
    # top_nhl_team_pct = round(team_points_by_nhl_team[0][1]/sum([row[1] for row in team_points_by_nhl_team])*100)
    # top_player = team_points_by_player[0]
    # top_player_pct = round(top_player["points"]/sum([row["points"] for row in team_points_by_player])*100)
    # top_player_by_team[team["team_key"]] = {"player_name": top_player["name"], "player_img": top_player["image_url"], "player_pct": top_player_pct}
top_opp_player_by_team = {k: sorted(v.values(), key=lambda x: x["points"], reverse=True)[0] for k, v in opp_players_by_team.items()}
top_opp_player_by_team_sorted = sorted(top_opp_player_by_team.items(), key=lambda x: x[1]["points"], reverse=True)
top_opp_player_by_team_sorted_ret = [{"rank": i+1, "image_url": v["image_url"], "main_text": v["name"], "sub_text": query.get_team_name_from_key(k), "stat": round(v["points"], 1)} for i, [k, v] in enumerate(top_opp_player_by_team_sorted)]
top_opp_player_by_team_sorted_ret
# top_player_by_team_sorted = sorted(top_player_by_team.items(), key=lambda x: x[1]["player_pct"], reverse=True)
# top_player_by_team_sorted_ret = [
#     {
#         "rank": i+1, 
#         "image_url": v["player_img"], 
#         "main_text": v["player_name"], 
#         "sub_text": query.get_team_name_from_key(k), 
#         "stat": v["player_pct"]
#     } for i, [k, v] in enumerate(top_player_by_team_sorted)
# ]
# top_player_by_team_sorted_ret

In [ ]:
def get_team_name(team_key):
    team_name = next(team["name"] for team in teams if team["team_key"] == team_key)
    return team_name

In [ ]:
def get_all_team_rosters_by_date(date):
    """
    Parameters:
    date (str): Date to get roster for in YYYY-MM-DD format

    Returns:
    List of teams with the roster details for that date
    (does not include stats, need to make a separate reequest to get stats for all these players)
    Trieed everything, can't get stats if you're getting data for multiple teams
    f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players/stats;type=date;date={date}" doesn't work
    """
    league_key = f"{game_id}.l.{league_id}"
    url = f"{BASE_URL}/league/{league_key}/teams/roster;date={date}/players" # TODO: Would it be better to store team_keys once and make request to teams endpoint instad of league?
    resp_data = api_request(url)["league"]["teams"]
    return resp_data

In [ ]:
def get_stats_for_players_by_date(player_keys, date):
    """
    Parameters:
    player_keys (str[]): All player keys
    date (str): Date to get roster for in YYYY-MM-DD format
    """
    league_key = f"{game_id}.l.{league_id}"
    data = []
    for i in range(math.ceil(len(player_keys)/25)):
        player_keys_str = ",".join(player_keys[i*25:min((i+1)*25, len(player_keys))])
        url = f"{BASE_URL}/league/{league_key}/players;player_keys={player_keys_str}/stats;type=date;date={date};start={i*25}" # Need to access league endpoint to get player_points attribute, otherwise calculate TODO: Is it faster to calculate it ourselves or request more information from the API?
        resp_data = api_request(url)
        data += resp_data["league"]["players"]
    return data

In [ ]:
def flatten_list_of_lists(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

In [ ]:
"""
Things to track
- Points missed out on
- Number of times roster wasn't optimized
"""
def get_bench_oppurtunity_cost(bench_players, team_roster, players_points):
    def get_replacement_players(bench_player):
        replacement_positions = bench_player["display_position"].split(",")
        replacement_players = []
        for position in replacement_positions:
            players_matching_position = [player for player in team_roster if player["selected_position"]["position"] == position and player["player_key"] not in selected_replacements]
            new_positions = flatten_list_of_lists([player["display_position"].split(",") for player in players_matching_position])
            new_positions_unique = [position for position in new_positions if position not in replacement_positions]
            replacement_positions += new_positions_unique
            replacement_players += players_matching_position
        return replacement_players

    selected_replacements = []
    points_missed_out = 0
    num_wrong_bench = 0
    for bench_player in sorted(bench_players, key=lambda player: players_points[player["player_key"]], reverse=True):
        replacement_players = get_replacement_players(bench_player)
        if len(replacement_players) == 0:
            num_wrong_bench += 1
            points_missed_out += players_points[bench_player["player_key"]] - players_points.get(worst_replacement_player["player_key"], 0)
            selected_replacements.append(worst_replacement_player["player_key"])
        else:
            worst_replacement_player = sorted(replacement_players, key=lambda player: players_points.get(player["player_key"], 0))[0]
            if players_points.get(worst_replacement_player["player_key"], 0) < players_points[bench_player["player_key"]]:
                num_wrong_bench += 1
                points_missed_out += players_points[bench_player["player_key"]] - players_points.get(worst_replacement_player["player_key"], 0)
                selected_replacements.append(worst_replacement_player["player_key"])
    num_benched = len(bench_players)
    return points_missed_out, num_wrong_bench, num_benched

In [ ]:
"""
Alternative reality
"""
# TODO: Check if playoff even exists, otherwise use endweek
num_reg_weeks = playoff_start_week - league_start_week
weeks = ",".join([str(week) for week in range(league_start_week, playoff_start_week)])
url = f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_key}/scoreboard;week={weeks}/matchups"
matchups = get_response(url)["league"]["scoreboard"]["matchups"]
team_schedules = defaultdict(lambda: {"points": [], "opponent": []})
for matchup in matchups:
    teams = matchup["teams"]
    team_schedules[teams[0]["team_key"]]["points"].append(float(teams[0]["team_points"]["total"]))
    team_schedules[teams[1]["team_key"]]["points"].append(float(teams[1]["team_points"]["total"]))
    team_schedules[teams[0]["team_key"]]["opponent"].append({"key": teams[1]["team_key"], "points": float(teams[1]["team_points"]["total"])})
    team_schedules[teams[1]["team_key"]]["opponent"].append({"key": teams[0]["team_key"], "points": float(teams[0]["team_points"]["total"])})
num_teams = 8 # TODO: change len(teams)
team_schedule_matrix = [[0 for _ in range(num_teams)] for _ in range(num_teams)]
# print(prettify_data(list(team_schedules.values())))
for i, [team_a_key, team_a]  in enumerate(team_schedules.items()):
    for j, team_b in enumerate(team_schedules.values()):
        results = [0.5 if team_a_pts == team_opp["points"] else team_a_pts > team_opp["points"] for team_a_pts, team_opp in zip(team_a["points"], team_b["opponent"]) if team_opp["key"] != team_a_key]
        percent = round(sum(results)/len(results), 3)
        team_schedule_matrix[i][j] = percent
team_schedule_matrix

In [ ]:
"""
Selected Position Values: {'G', 'NA', 'IR+', 'D', 'BN', 'RW', 'LW', 'C'}
- Number of games played by team
- Points left on bench

Theodore's Poo Poo Hospital 1066
Eric’s a Bitch Team         1045
Cyrus's Cursed Team         1043
eric's Awe-Inspiring Team   1021
Tim's Terrific Team         1004
adam2                       950
Kevin's Incredible Team     907
Miami Steamrollers          775
"""
league = get_league()
league_start_date = datetime.strptime(league["start_date"], "%Y-%m-%d")
league_end_date = datetime.strptime(league["end_date"], "%Y-%m-%d")
games_by_team = defaultdict(int)
points_left_on_bench_by_team = defaultdict(list)
benches_by_team = defaultdict(lambda: {"points_missed_out": 0, "num_wrong_bench": 0, "num_benched": 0})
for i in range((league_end_date - league_start_date).days+1):
    print(i)
    current_date_str = (league_start_date + timedelta(days=i)).strftime("%Y-%m-%d")
    teams = get_all_team_rosters_by_date(current_date_str)
    player_keys = [player["player_key"] for team in teams for player in team["roster"]["players"]] # This needs to be done first so request can be batched
    players = get_stats_for_players_by_date(player_keys, current_date_str)
    players_points = {player["player_key"]: float(player["player_points"]["total"]) for player in players if player["player_stats"]["stats"][0]["value"] != "-"} # Filter out players that didn't play that day
    for team in teams:
        bench_players = []
        team_roster = team["roster"]["players"]
        for player in team_roster:
            if player["selected_position"]["position"] not in ["BN", "IR+", "NA"] and player["player_key"] in players_points:
                games_by_team[team["team_key"]] += 1
            elif player["selected_position"]["position"] in ["BN", "IR+", "NA"] and player["player_key"] in players_points:
                bench_players.append(player)
        points_missed_out, num_wrong_bench, num_benched = get_bench_oppurtunity_cost(bench_players, team_roster, players_points)
        benches_by_team[team["team_key"]]["points_missed_out"] += points_missed_out
        benches_by_team[team["team_key"]]["num_wrong_bench"] += num_wrong_bench
        benches_by_team[team["team_key"]]["num_benched"] += num_benched
games_by_team_sorted = sorted(games_by_team.items(), key=lambda item: item[1], reverse=True)
games_by_team_sorted = [[get_team_name(team_key), games] for team_key, games in games_by_team_sorted]

In [ ]:
for team_key, bench_stats in benches_by_team.items():
    print(f'{get_team_name(team_key)} {round(bench_stats["points_missed_out"]/bench_stats["num_benched"], 2)} {round(bench_stats["num_wrong_bench"]/bench_stats["num_benched"], 2)}')

In [ ]:
def get_league_game_weeks():
    game_weeks = get_response(
        f"https://fantasysports.yahooapis.com/fantasy/v2/game/{game_id}/game_weeks", 
    )["game"]["game_weeks"]
    game_weeks_league = game_weeks
    game_weeks_league[league_start_week-1]["start"] = league_start_date_str # Should it be -1 or -league_start_week?
    game_weeks_league[league_end_week-1]["end"] = league_end_date_str
    return game_weeks_league

In [ ]:
def get_dates_by_week(week):
    league_game_weeks = get_league_game_weeks()
    start_date_str = league_game_weeks[week-1]["start"]
    end_date_str = league_game_weeks[week-1]["end"]
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    return [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

In [ ]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in get_response(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league['league_key']}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats"
        )["league"]["players"]
    ]

In [ ]:
def get_top_n_players_by_position(n, position):
    if position == "F":
        position = "C,LW,RW"
    return [
        player 
        for i in range(int(n/25)+1) 
        for player in get_response(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league['league_key']}/players;sort=PTS;sort_type=season;position={position};count={n};start={i*25}/stats",
        )["league"]["players"]
    ]

In [ ]:
def get_player_game_log_nhl(player_id):
    season = f"{league_season}{league_season+1}"
    return requests.get(f"https://api-web.nhle.com/v1/player/{player_id}/game-log/{season}/2").json()["gameLog"] # 2 inndicates regular season games

In [ ]:
def get_game_info_nhl(game_id):
    return requests.get(f"https://api-web.nhle.com/v1/gamecenter/{game_id}/landing").json()


In [ ]:
def get_player_team_by_date(player_name: str, player_position: str, date: str):
    '''
    Parameters:
    player_name (str): Full name of player
    date (str): Date to search for in YYYY-MM-DD format

    Returns:
    str: 3 letter abbreviation for NHL team name
    '''
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in player_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    game_log_for_date = next([game_log for game_log in player_game_log if game_log["gameDate"] == date])
    print(game_log_for_date)
    # return team_name

In [ ]:
'''
Most/least points after dropping a player on average
'''
transactions = query.get_league_transactions()
transactions_drop = [{"player": player, "timestamp": transaction.timestamp} for transaction in transactions for player in transaction.players if player.transaction_data.type == "drop"]
team_name = {}
points_by_team = defaultdict(list)
for transaction in transactions_drop:
    print(transaction)
    transaction_date = datetime.fromtimestamp(transaction["timestamp"], tz=ZoneInfo("America/New_York"))
    transaction_player = transaction["player"]
    team_key = transaction_player.transaction_data.source_team_key
    player_name = transaction_player.name.full
    player_name_url = player_name.replace(" ", "%20")
    players = requests.get(f"https://search.d3.nhle.com/api/v1/search/player?culture=en-us&limit=20&q={player_name_url}%2A").json()
    players = [player for player in players if player["name"] == player_name] # Matching names
    # If there are multiple players matching name, filter by other attributes
    if len(players) > 1:
        players = [player for player in players if player["lastSeasonId"] and int(player["lastSeasonId"][:4]) >= league_info.season]
        if len(players) > 1:
            players = [player for player in players if player["positionCode"] in transaction_player.display_position.split(",")]
    if len(players) != 1:
        print(len(players), player_name, players)
    player = players[0]
    player_game_log = get_player_game_log_nhl(player["playerId"])
    if len(player_game_log) == 0:
        print(f"{player_name} didn't play any games this season")
        continue
    # TODO: Consider case when someone is dropped due to injury
    # last_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() <= transaction_date.date()]
    # if len(last_game) == 0: # No games played before drop
    #     first_game = player_game_log[-1]
    #     print(f"{player_name} didn't play any games yet")
    #     continue
    # Next game could either be the same day as pickup or game after pickup
    next_game = [game_log for game_log in player_game_log if datetime.strptime(game_log["gameDate"], "%Y-%m-%d").replace(tzinfo=timezone(timedelta(hours=-4))).date() >= transaction_date.date()]
    if len(next_game) == 0: # Didn't play the rest of the season
        print(f"{player_name} didn't play any more games")
        continue
    next_game_date = next_game[-1]["gameDate"]
    player_stats = query.get_player_stats_by_date(transaction_player.player_key, next_game_date)
    points_by_team[team_key].append(player_stats.player_points.total)
    team_name[team_key] = transaction_player.transaction_data.source_team_name
team_average = []
for team_key, team_points in points_by_team.items():
    team_average.append((team_name[team_key], round(sum(team_points)/len(team_points), 1)))
team_average = sorted(team_average, key=lambda x: x[1])
print(team_average)

In [ ]:
'''
Biggest comeback
52.8s
'''
deficits = []
for week in range(league_start_week, league_end_week + 1):
    matchups = get_matchups_by_week(week)
    for matchup in matchups:
        # Comeback win can't happen without a winner
        if matchup.is_tied:
            continue
        winner_id = int(matchup.winner_team_key.split(".")[-1])
        team_ids = [matchup.teams[0].team_id, matchup.teams[1].team_id]
        team_w_idx, team_l_idx = (0, 1) if winner_id == team_ids[0] else (1, 0)
        deficit = 0
        start_date = datetime.strptime(matchup.week_start, "%Y-%m-%d")
        end_date = datetime.strptime(matchup.week_end, "%Y-%m-%d")
        # Don't include the last day since the matchup is over
        for i in range((end_date - start_date).days):
            current_date_str = (start_date + timedelta(days=i)).strftime("%Y-%m-%d")
            team_w_points = matchup.teams[team_w_idx].team_stats_collection[i]["team_points"].total
            team_l_points = matchup.teams[team_l_idx].team_stats_collection[i]["team_points"].total
            deficit = round(deficit + team_l_points - team_w_points, 1)
            if deficit > 0:
                deficits.append((deficit, matchup))
biggest_deficits = sorted(deficits, key=lambda x: x[0])
biggest_deficits = list({f"{matchup.week}.{matchup.winner_team_key}": (deficit, matchup) for (deficit, matchup) in biggest_deficits}.values())
biggest_deficits = sorted(biggest_deficits, reverse=True, key=lambda x: x[0])
print("Biggest comebacks")
for diff, matchup in biggest_deficits[:5]:
    print(f"{matchup.week} {matchup.teams[0].name} {matchup.teams[1].name} - {diff}")